In [4]:
import torch
import torch.nn as nn
import torch.onnx

import onnx
import onnxruntime as ort

import numpy as np

## Model for Test  
랜덤한 값으로 학습을 돌리고  
fix 한 다음에 테스트를 해보자  

In [53]:
class Model(nn.Module) :
    def __init__(self, n_channels = 10):
        super(Model,self).__init__()
        self.layer1 = nn.Conv2d(n_channels,n_channels,1)
        self.act = nn.GroupNorm(n_channels,n_channels)
        #self.act = nn.InstanceNorm2d(n_channels)
        self.layer2_1 = nn.Conv2d(n_channels,1,1)
        self.layer2_2 = nn.Conv2d(1,n_channels,1)
        self.layer2_3 = nn.Conv2d(n_channels,1,1)
        self.layer3 = nn.Linear(25,10)
        
    def forward(self,x):
        x = self.layer1(x)
        x = self.act(x)
        x = self.layer2_1(x)
        x = self.layer2_2(x)
        x = self.layer2_3(x)
        x = torch.reshape(x,(x.shape[0],25))
        x = self.layer3(x)
        
        return x
    
B = 2
C = 10
F = 5
T = 5


model = Model(n_channels = C)

optimizer = torch.optim.AdamW(model.parameters())
criterion = nn.MSELoss()

for i in range(100) :
    x = torch.rand(B,C,F,T)
    gt = torch.rand(B,10)
    y = model(x)
    loss = criterion(y,gt)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [58]:
model.eval()

## tracing
input = torch.rand(B,C,F,T)
torch_out = model(input.float())

with open('input.bin', 'wb') as f:
    np.save(f, input.detach().numpy())
with open('output_python.bin', 'wb') as f:
    np.save(f, torch_out.detach().numpy())

#torch.onnx.select_model_mode_for_export(model, torch.onnx.TrainingMode.EVAL)

dummy_input = torch.randn(input.shape)
torch.onnx.export(
model,         # model being run 
dummy_input,       # model input (or a tuple for multiple inputs) 
"export.onnx",       # where to save the model  
opset_version=16,
#do_constant_folding=False,
#keep_initializers_as_inputs=False,
input_names = ['input'], 
output_names = ['output'],
#dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
#'output' : {0 : 'batch_size'}},
#export_params=True,
#verbose = True,
#training = torch.onnx.TrainingMode.EVAL
)


""" previous setting
        torch.onnx.export(
            self,
            (dummy_input, dummy_states, dummy_pe_state),
            output_fp,
            verbose=False,
            opset_version=16,
            input_names=["inputs", "gru_state_in", "pe_state_in"],
            output_names=["outputs", "gru_state_out", "pe_state_out"],
        )

"""

#onnx_model = onnx.load("export.onnx")
#onnx.checker.check_model(onnx_model)

ort_session = ort.InferenceSession(
 "export.onnx",
 # providers=["CUDAExecutionProvider"]
  providers=["CPUExecutionProvider"]
)
x = input.numpy()
ort_inputs = {ort_session.get_inputs()[0].name: x}
ort_outs = ort_session.run(None, ort_inputs)

torch_out = torch_out.detach().numpy()

print(ort_outs)
print(torch_out)
np.testing.assert_allclose(torch_out, ort_outs[0], rtol=1e-05, atol=1e-05)

print("PASS!!")

[array([[0.20926006, 0.45192692, 0.5855003 , 0.6283846 , 0.4963988 ,
        0.4793894 , 0.55854   , 0.5268964 , 0.48668212, 0.4241364 ],
       [0.11567795, 0.5128454 , 0.4230445 , 0.507315  , 0.45039886,
        0.42831773, 0.4562284 , 0.4606209 , 0.4399253 , 0.38405535]],
      dtype=float32)]
[[0.20926003 0.45192698 0.5855003  0.62838453 0.49639893 0.4793894
  0.55854017 0.5268965  0.48668212 0.42413646]
 [0.11567806 0.5128454  0.4230445  0.50731504 0.45039886 0.42831773
  0.45622838 0.46062088 0.4399253  0.38405535]]
PASS!!
